<style>div.title-slide {    width: 100%;    display: flex;    flex-direction: row;            /* default value; can be omitted */    flex-wrap: nowrap;              /* default value; can be omitted */    justify-content: space-between;}</style><div class="title-slide">
<span style="float:left;">Licence CC BY-NC-ND</span>
<span>Thierry Parmentelat &amp; Arnaud Legout</span>
<span><img src="media/both-logos-small-alpha.png" style="display:inline" /></span>
</div>

# Les *property*

## Complément - niveau intermédiaire

Je vais considérer plusieurs implémentations possibles pour une classe  `Temperature`:

##### Implémentation naïve

In [ ]:
# dans sa version la plus épurée, une classe
# température pourrait ressembler à ça:

class Temperature1:
    def __init__(self, kelvin):
        self.kelvin = kelvin
        
    def __repr__(self):
        return f"{self.kelvin}°K"

In [ ]:
# créons une instance
t1 = Temperature1(20)
t1

In [ ]:
# et pour accéder à la valeur numérique je peux faire
t1.kelvin

Avec cette implémentation il est très facile de créer une température négative, qui n'a bien sûr pas de sens physique, ce n'est pas bon.

##### Interface *getter/setter*

Si vous avez été déjà exposés à des langages orientés objet comme C++, Java ou autre, vous avez peut-être l'habitude d'accéder aux données internes des instances par des **méthodes** de type *getter** ou **setter*, de façon à contrôler les accès et, dans une optique d'encapsulation, de préserver des invariants, comme ici le fait que la température doit être positive.

C'est-à-dire que vous vous dites peut-être, ça ne devrait pas être fait comme ça, on devrait plutôt proposer une interface pour accéder à l'implémentation interne; quelque chose comme:

In [ ]:
class Temperature2:
    def __init__(self, kelvin):
        # au lieu d'écrire l'attribut il est plus sûr
        # d'utiliser le setter
        self.set_kelvin(kelvin)
        
    def set_kelvin(self, kelvin):
        # je m'assure que _kelvin est toujours positif
        # et j'utilise un nom d'attribut avec un _ pour signifier
        # que l'attribut est privé et qu'il ne faut pas y toucher directement
        # on pourrait aussi bien sûr lever une exception 
        # mais ce n'est pas mon sujet ici
        self._kelvin = max(0, kelvin)
        
    def get_kelvin(self):
        return self._kelvin
        
    def __repr__(self):
        return f"{self._kelvin}°K"

Bon c'est vrai que d'un coté, c'est mieux parce que je garantis un invariant, la température est toujours positive:

In [ ]:
t2 = Temperature2(-30)
t2

Mais par contre, d'un autre coté, c'est très lourd, parce que chaque fois que je veux utiliser mon objet, je dois faire pour y accéder

In [ ]:
t2.get_kelvin()

et surtout, si j'avais déjà du code qui utilisait `t.kelvin` il va falloir le modifier entièrement.

##### Implémentation pythonique

La façon de s'en sortir ici consiste à définir une property. Comme on va le voir ce mécanisme permet d'écrire du code qui fait référence à l'attribut `kelvin` de l'instance, mais qui passe tout de même par une couche de logique.

Ça ressemblerait à ceci

In [ ]:
class Temperature3:
    def __init__(self, kelvin):
        self.kelvin = kelvin

    # je définis bel et bien mes accesseurs de type getter et setter
    # mais _get_kelvin commence avec un _ 
    # car il n'est pas censé être appelé par l'extérieur
    def _get_kelvin(self):
        return self._kelvin

    # idem
    def _set_kelvin(self, kelvin):
        self._kelvin = max(0, kelvin)
        
    # une fois que j'ai ces deux éléments je peux créer une property
    kelvin = property(_get_kelvin, _set_kelvin)
    
    # et toujours la façon d'imprimer
    def __repr__(self):
        return f"{self._kelvin}°K"    

In [ ]:
t3 = Temperature3(200)
t3

In [ ]:
# par contre ici on va le mettre à zéro
# à nouveau, une exception serait préférable sans doute
t3.kelvin = -30
t3

Comme vous pouvez le voir, cette technique a plusieurs avantages:
* on a ce qu'on cherchait, c'est-à-dire une façon d'ajouter une couche de logique lors des accès en lecture et en écriture à l'intérieur de l'objet,
* mais **sans toutefois** demander à l'utilisateur de passer son temps à envoyer des méthodes `get_` et `set()` sur l'objet, ce qui a tendance à alourdir considérablement le code.

C'est pour cette raison que vous ne rencontrerez presque jamais en python une librairie qui offre une interface à base de méthodes `get_something` et `set_something`, mais au contraire les API vous exposeront directement des attributs que vous devez utiliser directement.

##### Pour en savoir plus

Voir aussi [la documentation officielle](https://docs.python.org/3.6/library/functions.html#property)

Vous pouvez notamment aussi, en option, ajouter un *deleter* pour intercepter les instructions du type

In [ ]:
# comme on n'a pas défini de deleter, on ne peut pas faire ceci
try:
    del t3.kelvin
except Exception as e:
    print(f"OOPS {type(e)} {e}")